In [2]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout, Conv1D,GlobalMaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam


In [3]:

vocab_size = 20000
max_length = 500


(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)


x_train = pad_sequences(x_train, maxlen=max_length, padding='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post')


In [4]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=256, input_length=max_length),  
    Conv1D(128, 5, activation='relu'), 
    GlobalMaxPooling1D(),  
    Dropout(0.4),  
    Dense(128, activation='relu'),  
    Dropout(0.4),
    Dense(1, activation='sigmoid')
])

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [5]:
optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
early_stop = EarlyStopping(monitor='val_accuracy', patience=2, restore_best_weights=True)
print("Training model...")
history = model.fit(
    x_train, y_train,
    epochs=6,  
    batch_size=512, 
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=2
)

Training model...
Epoch 1/6
40/40 - 52s - 1s/step - accuracy: 0.5968 - loss: 0.6618 - val_accuracy: 0.6772 - val_loss: 0.5717
Epoch 2/6
40/40 - 57s - 1s/step - accuracy: 0.7742 - loss: 0.4870 - val_accuracy: 0.8410 - val_loss: 0.3617
Epoch 3/6
40/40 - 62s - 2s/step - accuracy: 0.8702 - loss: 0.3127 - val_accuracy: 0.8896 - val_loss: 0.2725
Epoch 4/6
40/40 - 63s - 2s/step - accuracy: 0.9330 - loss: 0.1830 - val_accuracy: 0.8942 - val_loss: 0.2662
Epoch 5/6
40/40 - 54s - 1s/step - accuracy: 0.9696 - loss: 0.0940 - val_accuracy: 0.8944 - val_loss: 0.2826
Epoch 6/6
40/40 - 67s - 2s/step - accuracy: 0.9864 - loss: 0.0471 - val_accuracy: 0.8922 - val_loss: 0.3322


In [6]:
loss, acc = model.evaluate(x_test, y_test, verbose=0)
print(f'\n Test Accuracy: {acc * 100:.2f}%')


 Test Accuracy: 88.27%


In [7]:
predictions = model.predict(x_test)
predicted_labels = (predictions > 0.5).astype(int).flatten()

782/782 ━━━━━━━━━━━━━━━━━━━━ 18s 23ms/step


In [8]:
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}

def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review if i > 2])

In [9]:
print("\nPredicted Results on Test Set:")
for i in range(10):
    if i == 8:
        continue
    print(f"\nReview {i+1}:")
    print(decode_review(x_test[i]))
    print(f"Predicted: {'Positive' if predicted_labels[i] == 1 else 'Negative'} | Actual: {'Positive' if y_test[i] == 1 else 'Negative'}")



Predicted Results on Test Set:

Review 1:
please give this one a miss br br kristy swanson and the rest of the cast rendered terrible performances the show is flat flat flat br br i don't know how michael madison could have allowed this one on his plate he almost seemed to know this wasn't going to work out and his performance was quite lacklustre so all you madison fans give this a miss
Predicted: Negative | Actual: Negative

Review 2:
this film requires a lot of patience because it focuses on mood and character development the plot is very simple and many of the scenes take place on the same set in frances austen's the sandy dennis character apartment but the film builds to a disturbing climax br br the characters create an atmosphere rife with sexual tension and psychological trickery it's very interesting that robert altman directed this considering the style and structure of his other films still the trademark altman audio style is evident here and there i think what really makes